In [2]:
# Install the litellm library - this is a tool that helps us talk to different AI models
!!pip install litellm

# IMPORTANT SETUP INSTRUCTIONS:
# You need to set your OpenAI API key as a secret in Google Colab
# Click the "key" icon on the left sidebar and add 'OPENAI_API_KEY' with your actual API key

# Import necessary libraries (think of these as toolboxes we need)
import os  # For working with the operating system and environment variables
from google.colab import userdata  # To get secret keys from Google Colab
import json  # For working with JSON data format (a way to structure information)
from typing import List  # For specifying what type of data functions should return
from litellm import completion  # The main tool to communicate with AI models

# Get the OpenAI API key from Google Colab's secret storage
api_key = userdata.get('OPENAI_API_KEY')
# Set this key as an environment variable so the AI can use it
os.environ['OPENAI_API_KEY'] = api_key

# ==================== TOOL FUNCTIONS ====================
# These are the "tools" or "skills" we're giving to our AI agent

def list_files() -> List[str]:
    """This function lists all files in the current directory (folder)."""
    return os.listdir(".")  # Get all files in the current folder and return them as a list

def read_file(file_name: str) -> str:
    """This function reads the contents of a specific file."""
    try:  # Try to do something, but be ready to handle errors
        with open(file_name, "r") as file:  # Open the file in read mode
            return file.read()  # Read all content from the file and return it
    except FileNotFoundError:  # If the file doesn't exist
        return f"Error: {file_name} not found."
    except Exception as e:  # If any other error happens
        return f"Error: {str(e)}"

def add_file(file_name: str, content: str, directory: str = ".") -> str:
    """This function creates a new file with specified content in a given directory."""
    try:
        # Create the full file path by joining directory and file name
        file_path = os.path.join(directory, file_name)

        # Create the directory if it doesn't exist
        os.makedirs(directory, exist_ok=True)

        # Write the content to the file
        with open(file_path, "w") as file:
            file.write(content)

        return f"Successfully created file '{file_name}' in directory '{directory}'"

    except PermissionError:
        return f"Error: Permission denied. Cannot write to directory '{directory}'"
    except Exception as e:
        return f"Error creating file: {str(e)}"

def terminate(message: str) -> None:
    """This function ends the conversation and prints a final message."""
    print(f"Termination message: {message}")

# ==================== TOOL SETUP ====================
# Create a dictionary that maps tool names to their actual functions
# This is like a phone book - when the AI says "call list_files", we know which function to run
tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "add_file": add_file,
    "terminate": terminate
}

# Define the tools in a format that the AI model can understand
# This is like giving the AI a manual of what each tool does and how to use it
tools = [
    {
        "type": "function",  # Tell the AI this is a function it can call
        "function": {
            "name": "list_files",  # The name of the tool
            "description": "Returns a list of files in the directory.",  # What it does
            "parameters": {"type": "object", "properties": {}, "required": []}  # No input needed
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},  # Needs a file name as input
                "required": ["file_name"]  # The file_name is required
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "add_file",
            "description": "Creates a new file with specified content in a given directory. If directory doesn't exist, it will be created.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {"type": "string", "description": "Name of the file to create (e.g., 'my_file.txt')"},
                    "content": {"type": "string", "description": "The text content to write in the file"},
                    "directory": {"type": "string", "description": "Directory path where to create the file (default is current directory '.')"}
                },
                "required": ["file_name", "content"]  # directory is optional, defaults to current directory
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},  # Needs a message as input
                },
                "required": ["message"]  # The message is required
            }
        }
    }
]

# ==================== AI AGENT INSTRUCTIONS ====================
# These are the rules and personality we give to our AI agent
agent_rules = [{
    "role": "system",  # This is a system message (like instructions for the AI)
    "content": """
You are an AI agent that can perform tasks by using available tools.
Available tools:
- list_files: Shows all files and directories in the current location
- read_file: Reads the content of any file
- add_file: Creates new files with custom content in specified directories

If a user asks about files, documents, or content, first list the files before reading them.

IMPORTANT: NEVER use the terminate tool unless the user explicitly tells you to "terminate" or "stop".
After completing any task, provide your results and ask what else you can help with.
Continue the conversation and wait for the user's next instruction.
Only call the terminate function if the user's message contains words like "terminate", "stop", "quit", "exit", or "done".
"""
}]

# ==================== MAIN PROGRAM SETUP ====================
# Initialize variables to control how the agent works
iterations = 0  # Keep track of how many times we've talked to the AI
max_iterations = 10  # Maximum number of conversations to prevent infinite loops

# Ask the user what they want the AI to do
user_task = input("What would you like me to do? ")

# Create a memory list to store the conversation history
# Start with the user's request
memory = [{"role": "user", "content": user_task}]

# ==================== THE MAIN AI AGENT LOOP ====================
# This loop keeps the conversation going until the task is done
while iterations < max_iterations:
    # Increment the iteration counter at the start of each loop
    iterations += 1

    # Combine the agent rules with the conversation memory
    messages = agent_rules + memory

    # Send everything to the AI model and get a response
    response = completion(
        model="openai/gpt-4o",  # Which AI model to use (GPT-4o from OpenAI)
        messages=messages,  # The conversation history
        tools=tools,  # The tools the AI can use
        max_tokens=1024  # Maximum length of the AI's response
    )

    # Check if the AI wants to use a tool
    if response.choices[0].message.tool_calls:
        # Extract information about which tool the AI wants to use
        tool = response.choices[0].message.tool_calls[0]  # Get the first (and only) tool call
        tool_name = tool.function.name  # What tool does it want to use?
        tool_args = json.loads(tool.function.arguments)  # What arguments/inputs for the tool?

        # Create a record of what action the AI wants to take
        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        # Handle the special "terminate" tool differently
        if tool_name == "terminate":
            print(f"Termination message: {tool_args['message']}")
            break  # Exit the loop - we're done!

        # If it's a tool we know about, execute it
        elif tool_name in tool_functions:
            try:
                # Call the actual function with the provided arguments
                # The ** syntax unpacks the arguments dictionary
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:  # If something goes wrong
                result = {"error": f"Error executing {tool_name}: {str(e)}"}

        # If the AI tried to use a tool we don't have
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        # Show what's happening (for debugging/monitoring)
        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")

        # Add this interaction to the conversation memory
        # First, record what the AI wanted to do
        # Then, record the result of doing it
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},
            {"role": "user", "content": json.dumps(result)}
        ])

    # If the AI doesn't want to use any tools, it's giving a regular text response
    else:
        result = response.choices[0].message.content  # Get the text response
        print(f"Response: {result}")

        # Add the AI's response to memory so it can continue the conversation
        memory.append({"role": "assistant", "content": result})

        # Ask for new user input to continue the conversation
        new_user_input = input("\nWhat else would you like me to do? (or type 'terminate' to stop): ")

        # Add the new user input to memory
        memory.append({"role": "user", "content": new_user_input})

        # Check if user wants to terminate
        if new_user_input.lower().strip() in ['terminate', 'stop', 'quit', 'exit', 'done']:
            print("Terminating as requested by user.")
            break

# ==================== END OF PROGRAM ====================
# The program ends here when either:
# 1. The AI calls the "terminate" tool
# 2. We reach the maximum number of iterations (safety measure)
#
# Note: The AI can now give text responses and continue working until it decides to terminate

What would you like me to do? list files
Executing: list_files with args {}
Result: {'result': ['.config', 'sample_data']}
Response: I found two directories: `.config` and `sample_data`. 

Is there anything specific you would like to do with these directories or their contents?

What else would you like me to do? (or type 'terminate' to stop): can you create a hello.txt file in the sample_data directory
Executing: add_file with args {'file_name': 'hello.txt', 'content': 'Hello, World!', 'directory': 'sample_data'}
Result: {'result': "Successfully created file 'hello.txt' in directory 'sample_data'"}
Response: The file `hello.txt` has been successfully created in the `sample_data` directory with the content "Hello, World!".

What would you like to do next?

What else would you like me to do? (or type 'terminate' to stop): list all the files in the sample_data directory
Executing: list_files with args {}
Result: {'result': ['.config', 'sample_data']}
Executing: list_files with args {}
Re